In [38]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Input
from IPython.display import display, HTML

import os

# Data Preprocessing
def preprocess_data(data):
    features = [
        'category', 'tags', 'location', 'opening_hours', 
        'family_friendly', 'cuisine_type', 'attraction_type', 'star_rating',
        'price_range', 'best_time_to_visit', 'peak_hours'
    ]
    data['combined_features'] = data[features].fillna('').apply(
        lambda x: ' '.join(x.map(str).str.lower().map(lambda s: re.sub(r'\W+', ' ', s))),
        axis=1
    )
    return data

# TF-IDF Matrix Creation
vectorizer = TfidfVectorizer()

def create_tfidf_matrix(data):
    tfidf_matrix = vectorizer.fit_transform(data['combined_features'])
    return tfidf_matrix

# ANN Data Preparation
def prepare_ann_data(data, tfidf_matrix):
    if 'target_column' not in data.columns:
        data['target_column'] = np.random.randint(0, 2, size=len(data))
    y = data['target_column']
    X = tfidf_matrix.todense()
    return train_test_split(X, y, test_size=0.2, random_state=42)

# Build and Train ANN
def train_ann(X_train, y_train):
    model = Sequential([
        Input(shape=(X_train.shape[1],)),  # Explicit Input layer
        Dense(128, activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(0.2),
        Dense(64, activation='relu', kernel_regularizer=l2(0.01)),
        Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    
    # Callbacks
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
    
    # Train the model
    model.fit(
        X_train, y_train, 
        epochs=50, batch_size=32, validation_split=0.2,
        callbacks=[early_stopping, reduce_lr]
    )
    
    # Save the model
    model.save('ann_model.keras')
    return model

# Model Evaluation
def evaluate_model(ann_model, X_test, y_test):
    y_pred = (ann_model.predict(X_test) > 0.5).astype(int)
    print("Model Evaluation:")
    print(classification_report(y_test, y_pred, zero_division=0))

# Recommendation System
def recommend_with_ann(input_text, data, tfidf_matrix, ann_model, top_n=10, category=None):
    input_vec = vectorizer.transform([input_text]).todense()
    predictions = ann_model.predict(tfidf_matrix)
    data['score'] = predictions.flatten()
    recommendations = data.sort_values(by='score', ascending=False).head(top_n)

    if category:
        recommendations = recommendations[recommendations['category'].str.contains(category, case=False, na=False)]
    
    return recommendations

# Handle multiple inputs
def process_user_input(user_input):
    # Split input by comma, strip whitespace, and combine into a single string
    return ' '.join([item.strip().lower() for item in user_input.split(',')])

# Display Recommendations
# Display Recommendations in Jupyter Notebook as a Table
def display_recommendations():
    # Load and preprocess dataset
    print("Loading dataset...")
    data = pd.read_csv('./data/mauritiusDataset.csv')
    data = preprocess_data(data)

    # TF-IDF Matrix
    print("Creating TF-IDF matrix...")
    tfidf_matrix = create_tfidf_matrix(data)

    # Train or load ANN
    if os.path.exists('ann_model.keras'):
        print("Loading existing ANN model...")
        ann_model = load_model('ann_model.keras')
    else:
        print("Training new ANN model...")
        X_train, X_test, y_train, y_test = prepare_ann_data(data, tfidf_matrix)
        ann_model = train_ann(X_train, y_train)
        evaluate_model(ann_model, X_test, y_test)

    # User input
    print("\nEnter preferences (comma-separated for multiple values):")
    amenity = input("Enter amenities (e.g., Free WiFi, Parking): ")
    attraction = input("Enter attractions (e.g., Historical site, Beach): ")
    category_filter = input("Enter a category to filter (optional, e.g., Restaurant, Hotel): ")

    # Process inputs
    amenity_text = process_user_input(amenity)
    attraction_text = process_user_input(attraction)
    input_text = f"{amenity_text} {attraction_text}"

    # Recommendations
    print("\nGenerating recommendations...")
    recommendations = recommend_with_ann(input_text, data, tfidf_matrix, ann_model, category=category_filter)

    # Display Required Columns in Notebook
    try:
        display_columns = ['name', 'address', 'url', 'rating', 'imageUrls', 'popularity_score', 'family_friendly', 'latitude', 'longitude']
        recommendations_table = recommendations[display_columns]

        # Display the table inline in the notebook
        print("\nTop Recommendations:")
        display(HTML(recommendations_table.to_html(index=False, escape=False, border=1)))
    except KeyError as e:
        print(f"Error: Missing required columns in recommendations: {e}")

# Execute the system
if __name__ == "__main__":
    display_recommendations()

Loading dataset...
Creating TF-IDF matrix...
Loading existing ANN model...

Enter preferences (comma-separated for multiple values):


Enter amenities (e.g., Free WiFi, Parking):  parking
Enter attractions (e.g., Historical site, Beach):  beach
Enter a category to filter (optional, e.g., Restaurant, Hotel):  hotel



Generating recommendations...
104/104 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step

Top Recommendations:


name,address,url,rating,imageUrls,popularity_score,family_friendly,latitude,longitude
Grants Villa,"QQ5P+3V2, Coastal Road, Trou d'Eau Douce, Mauritius",https://www.google.com/maps/search/?api=1&query=Grants%20Villa&query_place_id=ChIJqQBvwIz6fCERDt7EdFzCydQ,3.7,https://lh5.googleusercontent.com/p/AF1QipNUHsK1-WV463aEdSWTT70q4zwqHB-cTxlk5fwH=w426-h240-k-no,9.0,no,-20.242372,57.787149
Maya's friendly home,"HMC9+XP4, Plaine Magnien, Mauritius",https://www.google.com/maps/search/?api=1&query=Maya's%20friendly%20home&query_place_id=ChIJaYPpHwCLfCER1vgpiPUnHZ8,5.0,NaN,1.0,no,-20.427623,57.669328
residence le palmiste,NaN,NaN,3.5,NaN,6.0,no,-20.034586,57.544563
Villa Isora,"Calodyne, Mauritius",https://www.google.com/maps/search/?api=1&query=Villa%20Isora&query_place_id=ChIJjRfe2HWqfSERyxAyUZPyxKE,4.4,https://dynamic-media-cdn.tripadvisor.com/media/photo-o/08/67/e6/83/villa-azzurra.jpg?w=1900&h=1400&s=1,56.0,yes,-20.005841,57.642132
Cocotiers Hotel - Mauritius,"Tombeau Bay, Mauritius",https://www.google.com/maps/search/?api=1&query=Cocotiers%20Hotel%20-%20Mauritius&query_place_id=ChIJ6aYyXC1SfCERyivEHZCqAW0,3.6,https://lh5.googleusercontent.com/p/AF1QipPSoJSav168PTx1Yg3ztKYe_0LDUOFnWZ_D0z95=w408-h272-k-no,716.0,no,-20.110336,57.506570
